In [5]:
!pip install gurobipy
import numpy as np
from gurobipy import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Khởi tạo mô hình và trọng số

In [6]:
m = Model()
d = np.array([4, 3, 3, -4, 0, -6])
c = np.array([
    [0, 0, 0, 9, 6, 3],
    [0, 0, 0, 4, 6, 5],
    [0, 0, 0, 5, 8, 4],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0]
])
u = np.array([
    [0, 0, 0, 4, 5, 6],
    [0, 0, 0, 3, 2, 3],
    [0, 0, 0, 7, 4, 3],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 5, 0, 7],
    [0, 0, 0, 0, 0, 0]
])
d_vertex = len(d)

Khai báo biên

In [7]:
V = [i + 1 for i in range(d_vertex)]
E = [(i, j) for i in V for j in V if c[i - 1, j - 1] != 0]

a = m.addVars(V, vtype = GRB.CONTINUOUS, lb = -GRB.INFINITY, name = 'a')
b = m.addVars(E, vtype = GRB.CONTINUOUS, lb = 0, name = 'b')
x = m.addVars(E, vtype = GRB.BINARY, name = 'x')

Hàm mục tiêu

In [8]:
m.setObjective(4*a[1] + 3*a[2] + 3*a[3] - 4*a[4] -6*a[6] 
- 4*(1 - x[1, 4])*b[1, 4] - 5*(1 - x[1, 5])*b[1, 5] - 6*(1 - x[1, 6])*b[1, 6]
- 3*(1 - x[2, 4])*b[2, 4] - 2*(1 - x[2, 5])*b[2, 5] - 3*(1 - x[2, 6])*b[2, 6]
- 7*(1 - x[3, 4])*b[3, 4] - 4*(1 - x[3, 5])*b[3, 5] - 3*(1 - x[3, 6])*b[3, 6]
- 5*(1 - x[5, 4])*b[5, 4] - 7*(1 - x[5, 6])*b[5, 6], GRB.MAXIMIZE)

Khai báo các điều kiện ràng buộc

In [9]:
m.addConstrs(a[i] - a[j] - b[i, j] <= c[i - 1, j - 1] for (i, j) in E)
m.addConstr(quicksum(x[i, j]  for (i, j) in E) <= 2)
m.update()

In [10]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 12 rows, 28 columns and 44 nonzeros
Model fingerprint: 0x7b40c43e
Model has 11 quadratic objective terms
Variable types: 17 continuous, 11 integer (11 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  QObjective range [4e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 34 rows, 60 columns, 94 nonzeros
Presolved model has 22 SOS constraint(s)
Variable types: 38 continuous, 22 integer (22 binary)

Root relaxation: unbounded, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  postpo

In [11]:
m.printAttr('x')


    Variable            x 
-------------------------
        a[1]            9 
        a[2]            4 
        a[3]            3 
        a[6]           -1 
      b[1,5]            3 
      b[1,6]            7 
      x[1,5]            1 
      x[1,6]            1 
